In [1]:
from keras.models import Sequential
import time
from keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense, Activation,Dropout
from keras.models import Model
from utils import *
import glob
import numpy as np
from midi_parser import *
NOTE_SIZE = 156

Using TensorFlow backend.


In [3]:
drums, melodies = read_state_matrices()

In [3]:
def pad_track_list(stateList):
    trackList = stateList
    lengths = []
    for i in range(0, len(trackList)):
        lengths.append(len(trackList[i]))
    
    longest_track = lengths.index(max(lengths))
    max_length = max(lengths)
    
    for i in range(0, len(trackList)):
        zeros = np.zeros((max_length - len(trackList[i]), 156))
        trackList[i] = np.concatenate((trackList[i], zeros))
    return trackList

In [4]:
X = melodies
y = drums

In [6]:
np.shape(X_train)

(9,)

In [5]:
# split out a validation set
split_at = len(X) - len(X) / 10
X_train = X[:split_at]
X_val = X[split_at:]
y_train = y[:split_at]
y_val = y[split_at:]

In [16]:
X = []
y = []
window = 5
for j in range(len(X_train)):
    for i in range(1, len(X_train[j]), window):
        if (i + window >= len(X_train[j])):
            break
        X.append(X_train[j][i: i + window])
        y.append([y_train[j][i + window]])

#model = build_model(window, n_visible)
print np.shape(X)
X = np.array(X)
y = np.array(y)

(2472, 5, 156)


In [14]:
np.shape(X)

(2472, 5, 156)

In [9]:
NUM_NOTES = 156
HIDDEN_SIZE = 500 # arbitrary
MAXLEN = len(X[0]) # the maximum length of the training sample.
LAYERS = 1

In [10]:
model = Sequential()
model.add(LSTM(HIDDEN_SIZE,input_shape = (5,156))) # variable size
model.add(RepeatVector(1)) # input to decoder = hidden size thing repeated for each time step


for _ in range(LAYERS):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
   
    
model.add(TimeDistributed(Dense(NUM_NOTES)))
model.add(Activation('sigmoid'))


In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [12]:
model.fit(X, y, batch_size=100)

Epoch 1/10
2472/2472 [==============================] - 5s - loss: 6.5153 - acc: 0.0736     
Epoch 2/10
2472/2472 [==============================] - 3s - loss: 5.8588 - acc: 0.0886     
Epoch 3/10
2472/2472 [==============================] - 3s - loss: 5.5160 - acc: 0.1557     
Epoch 4/10
2472/2472 [==============================] - 3s - loss: 5.3173 - acc: 0.1553     
Epoch 5/10
2472/2472 [==============================] - 3s - loss: 5.2509 - acc: 0.1553     
Epoch 6/10
2472/2472 [==============================] - 3s - loss: 5.1490 - acc: 0.1557     
Epoch 7/10
2472/2472 [==============================] - 3s - loss: 5.0614 - acc: 0.1481     
Epoch 8/10
2472/2472 [==============================] - 3s - loss: 4.9981 - acc: 0.1343     
Epoch 9/10
2472/2472 [==============================] - 3s - loss: 4.9513 - acc: 0.1274     
Epoch 10/10
2472/2472 [==============================] - 3s - loss: 4.9016 - acc: 0.1525     


In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 500)               1314000   
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 1, 500)            0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 1, 500)            2002000   
_________________________________________________________________
time_distributed_7 (TimeDist (None, 1, 156)            78156     
_________________________________________________________________
activation_7 (Activation)    (None, 1, 156)            0         
Total params: 3,394,156
Trainable params: 3,394,156
Non-trainable params: 0
_________________________________________________________________


In [57]:
# test the generating
start = get_song(midiToStatematrix("./data/comrain_melody.mid")) # Start sequence for generated song

In [75]:
start_length = 20
generated = start[:start_length]
generated

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [76]:
while len(generated) < 300:
    x = np.expand_dims(generated[-5:], axis = 0)

    preds = model.predict(x, verbose=0)[0]
    next = sample(preds)

    generated = np.vstack((generated, next))

In [73]:
def sample(preds):
  # Helper function to sample an index from a probability array
  preds = np.asarray(preds).astype('float64')
  
  preds[preds == np.inf] = 0      
  return np.floor(preds + np.random.uniform(0, 1, preds.shape))

In [67]:
#generated = clean_song(generated)

In [77]:
statematrixToMidi(generated)

In [78]:
gen2 = clean_song(generated)

In [88]:
np.shape(generated)

(300, 156)